In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databrickseteproject167.dfs.core.windows.net/customers")
df = df.drop('_rescued_data')
df.display()

In [0]:
df = df.withColumn("domains", split(df.email, "@")[1])

In [0]:
df.groupBy("domains").agg(countDistinct("customer_id").alias("count_of_customers")).sort("count_of_customers", ascending=False).display()

In [0]:
df_email = df.filter(df.domains == "yahoo.com")
df_email.display()
time.sleep(10)
df_email = df.filter(df.domains == "gmail.com")
df_email.display()
time.sleep(10)
df_email = df.filter(df.domains == "hotmail.com")
df_email.display()
time.sleep(10)

In [0]:
df = df.withColumn("full name", concat(df.first_name, lit(" "), df.last_name))
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df = df.withColumnRenamed("full name", "full_name")
df.limit(20).display()

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databrickseteproject167.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databrickseteproject167_catalog.silver.customers
USING DELTA
LOCATION "abfss://silver@databrickseteproject167.dfs.core.windows.net/customers"